In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import xarray as xr

# Load the NetCDF files

In [ ]:
ds_precip = xr.open_dataset("/Users/james.a/Downloads/chirps_monthly_cleaned_06.nc")
ds_spi_gammma_03 = xr.open_dataset("/Users/james.a/git/climate_indices/trial_chirps7_spi_gamma_03.nc")
ds_spi_pearson_03 = xr.open_dataset("/Users/james.a/git/climate_indices/trial_chirps7_spi_pearson_03.nc")

# Inspect the dataset

In [ ]:
# Display variables, dimensions, coordinates
print(ds_precip)

In [ ]:
# select a time step by index
step = 6
time_step_precip = ds_precip["precip"].isel(time=step)
time_step_spi_gamma_03 = ds_spi_gammma_03["spi_gamma_03"].isel(time=step)
time_step_spi_pearson_03 = ds_spi_pearson_03["spi_pearson_03"].isel(time=step)

# create a map plot
fig = plt.figure(figsize=(10, 6))  # Set figure size
ax = plt.axes(projection=ccrs.PlateCarree())  # Use PlateCarree for simple lat/lon projection (common for grids)

time_step = time_step_spi_pearson_03

# Plot the 2D grid as a filled contour (or use .pcolormesh() for pixelated view)
time_step.plot.contourf(
    ax=ax,  # Attach to cartopy axis
    transform=ccrs.PlateCarree(),  # Ensure data is projected correctly
    cmap="viridis",  # Colormap (options: 'coolwarm', 'jet', 'RdBu_r', etc.)
    levels=20,  # Number of color levels (adjust for detail)
    add_colorbar=True,  # Automatically add colorbar
    cbar_kwargs={"label": "Temperature (°C)"},  # Customize colorbar
)

# Step 4: Add map features
ax.coastlines()  # Add coastlines
ax.add_feature(cfeature.BORDERS, linestyle=":")  # Add country borders
ax.add_feature(cfeature.LAND, edgecolor="black")  # Add land fill
ax.add_feature(cfeature.OCEAN)  # Add ocean fill
ax.gridlines(draw_labels=True)  # Add lat/lon gridlines with labels

# Customize the plot
plt.title("Temperature Map at Time: " + str(time_step.time.values))  # Dynamic title with time stamp
plt.show()

In [ ]:
from IPython.display import HTML
from matplotlib.animation import FuncAnimation

# Select the variable to animate (e.g., spi_pearson_03; you can change this to another one)
data_var = ds_spi_pearson_03["spi_pearson_03"]  # Full DataArray with all time steps
num_steps = data_var.sizes["time"]  # Number of time steps

# Create the figure and axis with cartopy projection
fig = plt.figure(figsize=(10, 6))
ax = plt.axes(projection=ccrs.PlateCarree())

# Add static map features (these don't change per frame)
ax.coastlines()
ax.add_feature(cfeature.BORDERS, linestyle=":")
ax.add_feature(cfeature.LAND, edgecolor="black")
ax.add_feature(cfeature.OCEAN)
ax.gridlines(draw_labels=True)

# Initialize the plot with the first time step using pcolormesh (easier to update than contourf)
time_step_init = data_var.isel(time=0)
im = time_step_init.plot.pcolormesh(
    ax=ax,
    transform=ccrs.PlateCarree(),
    cmap="viridis",  # Colormap
    add_colorbar=False,  # We'll add a static colorbar below
)

# Add a static colorbar (based on the full data range for consistency)
vmin = data_var.min().values  # Min value across all time steps
vmax = data_var.max().values  # Max value across all time steps
cbar = plt.colorbar(im, ax=ax, label="SPI Pearson 03", extend="both")  # Adjust label as needed
im.set_clim(vmin, vmax)  # Set consistent color limits

# Set initial title
title = ax.set_title("Map at Time: " + str(time_step_init.time.values))


# Define the update function for each frame
def update(frame):
    # Select the current time step
    time_step = data_var.isel(time=frame)

    # Update the image data (pcolormesh array)
    im.set_array(time_step.values.ravel())  # Flatten the 2D array for pcolormesh update

    # Update the title with the current time
    title.set_text("Map at Time: " + str(time_step.time.values))

    return [im, title]


# Create the animation
anim = FuncAnimation(
    fig,
    update,
    frames=range(num_steps),  # Loop over all time steps
    interval=500,  # 500 ms = 0.5 seconds per frame
    repeat=True,  # Loop indefinitely
    blit=True,  # Optimize by only redrawing changed elements
)

# Display the animation in Jupyter notebook
HTML(anim.to_jshtml())